In [1]:
import itertools
import re

import gffutils
import numpy as np
import pandas as pd
import pybedtools
import six

v19db_filename = '/home/obotvinnik/scratch/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)
bed_folder = '{}/bed'.format(folder)



splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)

splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)
six.print_(splicing_feature_data.shape)
splicing_feature_data.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(36531, 26)


,Unnamed: 0,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,...,junction12,junction13,junction23,junction24,junction34,strand,transcript_id,exon2_divisible_by_3,one_ensembl_id,splice_type
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,...,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",True,ENSG00000107521,SE
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,NaN,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,...,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",False,ENSG00000107521,SE
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,NaN,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,...,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,"ENST00000361490.4,ENST00000325103.6,ENST000004...",False,ENSG00000107521,SE
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,...,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,"ENST00000543866.1,ENST00000370508.5",False,ENSG00000120053,SE
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,NaN,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,...,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,ENST00000370489.4,False,ENSG00000198018,SE


In [2]:
splicing_feature_data.columns

Index([u'Unnamed: 0', u'criteria', u'criteria_additional', u'criteria_full',
       u'ensembl_id', u'exon1', u'exon1_length', u'exon2', u'exon2_length',
       u'exon3', u'exon3_length', u'exon4', u'exon4_length', u'gencode_id',
       u'gene_name', u'intron_length', u'junction12', u'junction13',
       u'junction23', u'junction24', u'junction34', u'strand',
       u'transcript_id', u'exon2_divisible_by_3', u'one_ensembl_id',
       u'splice_type'],
      dtype='object')

In [3]:
constitutive = pd.read_table(constitutive_bedfile, header=None)
constitutive.head()

,0,1,2,3,4,5
0,chr10,102039881,102039985,exon:chr10:102039881-102039985:-,1000,-
1,chr10,102040691,102040810,exon:chr10:102040691-102040810:-,1000,-
2,chr10,102045854,102045970,exon:chr10:102045854-102045970:-,1000,-
3,chr10,102286732,102286831,exon:chr10:102286732-102286831:-,1000,-
4,chr10,102289123,102289263,exon:chr10:102289123-102289263:-,1000,-


In [4]:
exon_i = None

def get_first_last_internal(exon_id, db):
    exon = db[exon_id]

    first_internals = []
    last_internals = []
    for transcript in db.parents(exon, featuretype='transcript'):
        children = db.children(transcript, featuretype='exon', 
                                  order_by='start', reverse=exon.strand=='-')
        for i, child in enumerate(children):
            if child == exon:
                exon_i = i
        first_internal = (exon_i == 1)
        last_internal = (exon_i == (i - 1))
        first_internals.append(first_internal)
        last_internals.append(last_internal)
#     return (('last_internal', any(last_internals)), ('first_internal', any(first_internals)))
    return any(last_internals), any(first_internals)

In [ ]:
csv = 'first_last_internal.csv'

## Constitutive

In [ ]:
%%time
constitutive_first_last_internal = constitutive[3].map(lambda x: get_first_last_internal(x, db=v19db))

In [ ]:
constitutive_first_last_internal.index = constitutive[3]

In [ ]:
constitutive_first_last_internal.head()

In [ ]:
x = constitutive_first_last_internal.map(list).to_dict()
constitutive_first_last_internal_df = pd.DataFrame(x).T

In [ ]:

constitutive_first_last_internal_df.to_csv('{}/{}'.format(constitutive_feature_folder, csv))

## Alternative

In [ ]:
%%time
first_last_internal = splicing_feature_data['exon2'].apply(get_first_last_internal, db=v19db)

In [ ]:
# first_last_internal.loc[first_last_internal.all(axis=1)]

In [ ]:
# range(3)

In [ ]:
x = first_last_internal.map(list).to_dict()
first_last_internall_df = pd.DataFrame(x).T

In [ ]:
first_last_internal.to_csv('{}/first_last_internal.csv'.format(alternative_feature_folder))

In [ ]:
range(3)